## Contributions

The following are the contributions made to the original code:
 * Implemented GPU parallelism. Added a _DistributedSampler_ as an alternative to _Sampler_.
 * Wrapped the model into a _DataParallel_.
 * Made modifications to the _collatebatch_ in order to pad vectors for multi-GPU tensor processing.
 * Managed to sucesfully train on 8 NVIDIA V40.

Due to the simplicity of the model, the training on multiple GPUs actually caused slower progress. The *second portion* of this notebook proceeds to train on a single GPU only.

# Multi-GPU training: setup of Data structures

In [1]:
import os
import json
import torch
import random
import math
import torch.nn as nn
import torch.distributed as dist
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Optimizer
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data.distributed import DistributedSampler
from pathlib import Path
from tqdm import tqdm


class myDataset(Dataset):
  def __init__(self, data_dir, segment_len=128):
    self.data_dir = data_dir
    self.segment_len = segment_len

    # Load the mapping from speaker name to their corresponding id.
    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())
    self.speaker2id = mapping["speaker2id"]

    # Load metadata of training data.
    metadata_path = Path(data_dir) / "metadata.json"
    metadata = json.load(open(metadata_path))["speakers"]

    # Get the total number of speakers.
    self.speaker_num = len(metadata.keys())
    self.data = []
    for speaker in metadata.keys():
      for utterances in metadata[speaker]:
        self.data.append([utterances["feature_path"], self.speaker2id[speaker]])

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    feat_path, speaker = self.data[index]
    # Load preprocessed mel-spectrogram.
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    # Segment mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start + self.segment_len])
    else:
      mel = torch.FloatTensor(mel)
    # Turn the speaker id into long for computing loss later.
    speaker = torch.FloatTensor([speaker]).long()
    return mel, speaker

  def get_speaker_number(self):
    return self.speaker_num


def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data."""
  mel, speaker = zip(*batch)
  # We train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)  # pad log 10^(-20) which is very small value.
  # mel: (batch size, length, 40)
  return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
    dataset = myDataset(data_dir)
    speaker_num = dataset.get_speaker_number()
    trainlen = int(0.9 * len(dataset))
    lengths = [trainlen, len(dataset) - trainlen]
    trainset, validset = random_split(dataset, lengths)

    train_sampler = DistributedSampler(trainset)
    valid_sampler = DistributedSampler(validset)

    train_loader = DataLoader(
        trainset,
        batch_size=batch_size,
        sampler=train_sampler,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    valid_loader = DataLoader(
        validset,
        batch_size=batch_size,
        sampler=valid_sampler,
        num_workers=n_workers,
        pin_memory=True,
        collate_fn=collate_batch,
    )
    return train_loader, valid_loader, speaker_num


class Classifier(nn.Module):
  def __init__(self, d_model=80, n_spks=600, dropout=0.1):
    super().__init__()
    # Project the dimension of features from that of input into d_model.
    self.prenet = nn.Linear(40, d_model)
    # TODO:
    #   Change Transformer to Conformer.
    #   https://arxiv.org/abs/2005.08100
    self.encoder_layer = nn.TransformerEncoderLayer(
      d_model=d_model, dim_feedforward=128, nhead=2
    )
    # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

    # Project the the dimension of features from d_model into speaker nums.
    self.pred_layer = nn.Sequential(
      nn.Linear(d_model, d_model),
      nn.ReLU(),
      nn.Linear(d_model, n_spks),
    )

  def forward(self, mels):
    """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
    # out: (batch size, length, d_model)
    out = self.prenet(mels)
    # out: (length, batch size, d_model)
    out = out.permute(1, 0, 2)
    # The encoder layer expect features in the shape of (length, batch size, d_model).
    out = self.encoder_layer(out)
    # out: (batch size, length, d_model)
    out = out.transpose(0, 1)
    # mean pooling
    stats = out.mean(dim=1)

    # out: (batch, n_spks)
    out = self.pred_layer(stats)
    return out
  

def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,
  num_training_steps: int,
  num_cycles: float = 0.5,
  last_epoch: int = -1,
):

  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)


def model_fn(batch, model, criterion, device):
    """Forward a batch through the model."""

    mels, labels = batch
    mels = mels.to(device)
    labels = labels.to(device)
    labels = labels.flatten()  # Flatten labels to 1D

    outs = model(mels)

    loss = criterion(outs, labels)

    # Get the speaker id with highest probability.
    preds = outs.argmax(1)
    # Compute accuracy.
    accuracy = torch.mean((preds == labels).float())

    return loss, accuracy

def valid(dataloader, model, criterion, device): 
  """Validate on validation set."""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.2f}",
      accuracy=f"{running_accuracy / (i+1):.2f}",
    )

  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)

# Main function

In [2]:
import time

def parse_args():
    """Arguments"""
    config = {
        "data_dir": "./Dataset",
        "save_path": "model.ckpt",
        "batch_size": 256,
        "n_workers": 0,
        "valid_steps": 2000,
        "warmup_steps": 1000,
        "save_steps": 4000,
        "total_steps": 70000,
    }
    return config


def main_multi_gpu(data_dir, save_path, batch_size, n_workers, valid_steps, warmup_steps, total_steps, save_steps):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    dataset = myDataset(data_dir)
    train_loader = DataLoader(dataset, batch_size=batch_size, num_workers=n_workers)

    model = Classifier(d_model=80, n_spks=600, dropout=0.1).to(device)

    if torch.cuda.device_count() > 1:
        print("Using", torch.cuda.device_count(), "GPUs")
        model = nn.DataParallel(model)

    criterion = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=1e-3)
    scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)

    best_accuracy = -1.0
    best_state_dict = None

    pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")
    time_limit = 120  # 2 minute in seconds

    for step in range(total_steps):
        batch = next(iter(train_loader))
        loss, accuracy = model_fn(batch, model, criterion, device)

        if step == 0:
            start_time = time.time()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        elapsed_time = time.time() - start_time
        estimated_time_left = (total_steps - step) * (elapsed_time / (step+1))
        if step>30 and estimated_time_left > time_limit:
            print(f'Stopping training: epoch is taking too long compared to single GPU ({int(estimated_time_left)} > {time_limit} sec).')
            return

        pbar.update()
        pbar.set_postfix(loss=f"{loss.item():.2f}", accuracy=f"{accuracy.item():.2f}", step=step + 1)

        if (step + 1) % valid_steps == 0:
            pbar.close()
            valid_accuracy = valid(train_loader, model, criterion, device)

            if valid_accuracy > best_accuracy:
                best_accuracy = valid_accuracy
                best_state_dict = model.state_dict()
                pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

        if (step + 1) % save_steps == 0 and best_state_dict:
            torch.save(best_state_dict, save_path)
            pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")
    pbar.close()

main_multi_gpu(**parse_args())

Using 8 GPUs


Train:   2% 31/2000 [00:14<15:51,  2.07 step/s, accuracy=0.24, loss=6.16, step=31]

Stopping training: epoch is taking too long compared to single GPU (7888 > 120 sec).


# Single GPU training

As no significant speedup was observed, we proceed with single GPU trianing. We redefine some important functions for single GPU.

In [2]:
def get_dataloader(data_dir, batch_size, n_workers):
  """Generate dataloader"""
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # Split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)
  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  return train_loader, valid_loader, speaker_num

def model_fn(batch, model, criterion, device):
  """Forward a batch through the model."""

  mels, labels = batch
  mels = mels.to(device)
  labels = labels.to(device)

  outs = model(mels)

  loss = criterion(outs, labels)

  # Get the speaker id with highest probability.
  preds = outs.argmax(1)
  # Compute accuracy.
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy

### Main single GPU

In [ ]:
def parse_args():
  """arguments"""
  config = {
    "data_dir": "./Dataset",
    "save_path": "model.ckpt",
    "batch_size": 32,
    "n_workers": 0,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 4000,
    "total_steps": 70000,
  }

  return config


def main(
  data_dir,
  save_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  train_iterator = iter(train_loader)
  print(f"[Info]: Finish loading data!",flush = True)

  model = Classifier(d_model=80, n_spks=speaker_num).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = AdamW(model.parameters(), lr=1e-3)
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[Info]: Finish creating model!",flush = True)

  best_accuracy = -1.0
  best_state_dict = None

  pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

  for step in range(total_steps):
    # Get data
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)

    loss, accuracy = model_fn(batch, model, criterion, device)
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()

    # Updata model
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    
    # Log
    pbar.update()
    pbar.set_postfix(
      loss=f"{batch_loss:.2f}",
      accuracy=f"{batch_accuracy:.2f}",
      step=step + 1,
    )

    # Do validation
    if (step + 1) % valid_steps == 0:
      pbar.close()

      valid_accuracy = valid(valid_loader, model, criterion, device)

      # keep the best model
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()

      pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    # Save the best model so far.
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      save_path = f"model_{step + 1}_{best_accuracy:.4f}.ckpt"
      torch.save(best_state_dict, save_path)
      pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

  pbar.close()


if __name__ == "__main__":
  main(**parse_args())

Train:   4% 75/2000 [01:17<33:08,  1.03s/ step, accuracy=0.01, loss=6.40, step=75]
Train:  20% 403/2000 [00:35<02:21, 11.25 step/s, accuracy=0.00, loss=5.98, step=403]

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!



Train:   9% 174/2000 [00:09<01:34, 19.27 step/s, accuracy=0.00, loss=6.32, step=174]

Train: 100% 2000/2000 [01:36<00:00, 20.64 step/s, accuracy=0.06, loss=4.44, step=2000][A
Valid: 100% 4832/4836 [00:07<00:00, 606.54 uttr/s, accuracy=0.20, loss=3.85]
Train: 100% 2000/2000 [01:14<00:00, 26.88 step/s, accuracy=0.34, loss=3.10, step=4000]
Valid: 100% 4832/4836 [00:05<00:00, 899.98 uttr/s, accuracy=0.31, loss=3.21]
Train:   0% 0/2000 [00:00<?, ? step/s]

                                                                                  

Train:   0% 5/2000 [00:00<01:21, 24.42 step/s, accuracy=0.22, loss=3.07, step=4005][A

Step 4000, best model saved. (accuracy=0.3094)


Train: 100% 2000/2000 [01:14<00:00, 26.97 step/s, accuracy=0.28, loss=3.05, step=6000]
Valid: 100% 4832/4836 [00:05<00:00, 901.77 uttr/s, accuracy=0.37, loss=2.87]
Train: 100% 2000/2000 [01:13<00:00, 27.19 step/s, accuracy=0.34, loss=3.03, step=8000]
Valid: 100% 4832/4836 [00:05<00:00, 906.15 uttr/s, accuracy=0.40, loss=2.74]
Train:   0% 0/2000 [00:00<?, ? step/s]

                                                                                  

Train:   0% 5/2000 [00:00<01:17, 25.70 step/s, accuracy=0.31, loss=3.21, step=8005][A

...

Train: 100% 2000/2000 [01:13<00:00, 27.24 step/s, accuracy=0.47, loss=1.58, step=3e+4] 
Valid: 100% 4832/4836 [00:05<00:00, 904.16 uttr/s, accuracy=0.57, loss=1.90]
Train: 100% 2000/2000 [01:13<00:00, 27.30 step/s, accuracy=0.59, loss=1.81, step=32000]
Valid: 100% 4832/4836 [00:05<00:00, 916.29 uttr/s, accuracy=0.57, loss=1.91]
                                      

Train:   0% 0/2000 [00:00<?, ? step/s]                                            

Train:   0% 5/2000 [00:00<01:18, 25.44 step/s, accuracy=0.59, loss=1.74, step=32005]A

Step 32000, best model saved. (accuracy=0.5712)


Train: 100% 2000/2000 [01:13<00:00, 27.33 step/s, accuracy=0.62, loss=1.36, step=34000]
Valid: 100% 4832/4836 [00:05<00:00, 908.55 uttr/s, accuracy=0.57, loss=1.91]
Train: 100% 2000/2000 [01:13<00:00, 27.15 step/s, accuracy=0.62, loss=1.54, step=36000]
Valid: 100% 4832/4836 [00:05<00:00, 890.03 uttr/s, accuracy=0.58, loss=1.86]
Train:   0% 0/2000 [00:00<?, ? step/s]

                                                                                  

Train:   0% 5/2000 [00:00<01:22, 24.28 step/s, accuracy=0.62, loss=1.42, step=36005]A

Step 36000, best model saved. (accuracy=0.5753)


Train:  32% 632/2000 [00:23<00:51, 26.59 step/s, accuracy=0.66, loss=1.49, step=36632]

## Dataset of inference

In [3]:
import os
import json
import torch
from pathlib import Path
from torch.utils.data import Dataset


class InferenceDataset(Dataset):
  def __init__(self, data_dir):
    testdata_path = Path(data_dir) / "testdata.json"
    metadata = json.load(testdata_path.open())
    self.data_dir = data_dir
    self.data = metadata["utterances"]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    utterance = self.data[index]
    feat_path = utterance["feature_path"]
    mel = torch.load(os.path.join(self.data_dir, feat_path))

    return feat_path, mel


def inference_collate_batch(batch):
  """Collate a batch of data."""
  feat_paths, mels = zip(*batch)

  return feat_paths, torch.stack(mels)


## Main funcrion of Inference

In [9]:
import json
import csv
import glob
from pathlib import Path
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

def parse_args():
  """arguments"""
  config = {
    "data_dir": "./Dataset",
    "model_dir": ".",  # directory where model files are saved
    "output_path": "./output.csv",
  }

  return config


def main(
  data_dir,
  model_dir,
  output_path,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  mapping_path = Path(data_dir) / "mapping.json"
  mapping = json.load(mapping_path.open())

  dataset = InferenceDataset(data_dir)
  dataloader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    num_workers=0,
    collate_fn=inference_collate_batch,
  )
  print(f"[Info]: Finish loading data!",flush = True)

  speaker_num = len(mapping["id2speaker"])
  model = Classifier(n_spks=speaker_num).to(device)

  # new code to automatically find and load the best model
  list_of_files = glob.glob(f"{model_dir}/model_*.ckpt")  # get list of all model files
  latest_file = max(list_of_files, key=lambda x: float(x.split('_')[2].split('.')[0]))  # get the best model file
  model.load_state_dict(torch.load(latest_file))
  
  model.eval()
  print(f"[Info]: Finish creating model!",flush = True)

  results = [["Id", "Category"]]
  for feat_paths, mels in tqdm(dataloader):
    with torch.no_grad():
      mels = mels.to(device)
      outs = model(mels)
      preds = outs.argmax(1).cpu().numpy()
      for feat_path, pred in zip(feat_paths, preds):
        results.append([feat_path, mapping["id2speaker"][str(pred)]])
  
  with open(output_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(results)


if __name__ == "__main__":
  main(**parse_args())

[Info]: Use cuda now!
[Info]: Finish loading data!
[Info]: Finish creating model!




  0%|                                                                                               | 0/6657 [00:00<?, ?it/s]

  1%|▋                                                                                    | 52/6657 [00:00<00:12, 518.29it/s]

  2%|█▎                                                                                  | 104/6657 [00:00<00:13, 499.46it/s]

  2%|█▉                                                                                  | 154/6657 [00:00<00:13, 497.79it/s]

  3%|██▌                                                                                 | 206/6657 [00:00<00:12, 504.95it/s]

  4%|███▏                                                                                | 257/6657 [00:00<00:12, 496.85it/s]

  5%|███▉                                                                                | 309/6657 [00:00<00:12, 503.06it/s]

  5%|████▌                                                                               | 360/6657 [00:00<00

 48%|████████████████████████████████████████                                           | 3209/6657 [00:06<00:06, 506.27it/s]

 49%|████████████████████████████████████████▋                                          | 3261/6657 [00:06<00:06, 507.25it/s]

 50%|█████████████████████████████████████████▎                                         | 3312/6657 [00:06<00:06, 502.27it/s]

 51%|█████████████████████████████████████████▉                                         | 3363/6657 [00:06<00:06, 497.67it/s]

 51%|██████████████████████████████████████████▌                                        | 3415/6657 [00:06<00:06, 501.12it/s]

 52%|███████████████████████████████████████████▏                                       | 3466/6657 [00:06<00:06, 501.69it/s]

 53%|███████████████████████████████████████████▉                                       | 3519/6657 [00:06<00:06, 509.73it/s]

 54%|████████████████████████████████████████████▌                                      | 3573/6657 [00:06<00:0

 77%|████████████████████████████████████████████████████████████████▏                  | 5149/6657 [00:13<00:07, 214.68it/s]

 78%|████████████████████████████████████████████████████████████████▍                  | 5171/6657 [00:13<00:06, 214.80it/s]

 78%|████████████████████████████████████████████████████████████████▋                  | 5193/6657 [00:13<00:06, 212.81it/s]

 78%|█████████████████████████████████████████████████████████████████                  | 5215/6657 [00:13<00:06, 210.65it/s]

 79%|█████████████████████████████████████████████████████████████████▎                 | 5237/6657 [00:13<00:06, 212.87it/s]

 79%|█████████████████████████████████████████████████████████████████▌                 | 5260/6657 [00:13<00:06, 216.13it/s]

 79%|█████████████████████████████████████████████████████████████████▊                 | 5282/6657 [00:13<00:06, 209.44it/s]

 80%|██████████████████████████████████████████████████████████████████▏                | 5304/6657 [00:13<00:0

 97%|████████████████████████████████████████████████████████████████████████████████▋  | 6467/6657 [00:19<00:00, 205.30it/s]

 97%|████████████████████████████████████████████████████████████████████████████████▉  | 6489/6657 [00:19<00:00, 208.06it/s]

 98%|█████████████████████████████████████████████████████████████████████████████████▏ | 6510/6657 [00:19<00:00, 207.46it/s]

 98%|█████████████████████████████████████████████████████████████████████████████████▍ | 6532/6657 [00:19<00:00, 211.09it/s]

 98%|█████████████████████████████████████████████████████████████████████████████████▋ | 6554/6657 [00:19<00:00, 213.49it/s]

 99%|█████████████████████████████████████████████████████████████████████████████████▉ | 6576/6657 [00:19<00:00, 209.49it/s]

 99%|██████████████████████████████████████████████████████████████████████████████████▎| 6597/6657 [00:19<00:00, 207.82it/s]

 99%|██████████████████████████████████████████████████████████████████████████████████▌| 6618/6657 [00:20<00:0